In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, trim_messages

llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
messages = [
  SystemMessage(content="System"),
  HumanMessage(content="Human1"),
  AIMessage(content="AI1"),
  HumanMessage(content="Human2"),
  AIMessage(content="AI2"),
  HumanMessage(content="Human3"),
  AIMessage(content="AI3"),
]

In [ ]:
# 대화 많이 남기기
trimmer = trim_messages(
  max_tokens=1000,
  token_counter=llm
)

trimmer.invoke(messages)

[SystemMessage(content='System', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human1', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI1', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI2', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [5]:
# 대화 적게 남기기
trimmer = trim_messages(
  max_tokens=20,
  token_counter=llm
)

trimmer.invoke(messages)

[HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [6]:
# 과거 대화 남기고 최근 대화 지우기
trimmer = trim_messages(
  max_tokens=20,
  token_counter=llm,
  strategy="first"
)

trimmer.invoke(messages)

[SystemMessage(content='System', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Human1', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI1', additional_kwargs={}, response_metadata={})]

In [7]:
# 과거 대화 지우고 최근 대화 남기기
trimmer = trim_messages(
  max_tokens=20,
  token_counter=llm,
  strategy="last"
)

trimmer.invoke(messages)

[HumanMessage(content='Human3', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [8]:
# 과거 대화 지우고 최근 대화 남기기s는데 시스템 메시지는 남기기
trimmer = trim_messages(
  max_tokens=20,
  token_counter=llm,
  strategy="last",
  include_system=True
)

trimmer.invoke(messages)

[SystemMessage(content='System', additional_kwargs={}, response_metadata={}),
 AIMessage(content='AI3', additional_kwargs={}, response_metadata={})]

In [9]:
# 과거 대화 지우고 최근 대화 남기기s는데 시스템 메시지도 남기고 HumanMessage 부터 시작하기
trimmer = trim_messages(
  max_tokens=20,
  token_counter=llm,
  strategy="last",
  include_system=True,
  start_on="human"
)

trimmer.invoke(messages)

[SystemMessage(content='System', additional_kwargs={}, response_metadata={})]

In [10]:
messages = [
  SystemMessage(content="사용자의 질문에 2문장 이내로 짧게 대답해"),
  HumanMessage(content="오늘은 떡볶이를 먹어야지!"),
  AIMessage(content="정말 좋은 생각이야. 음료는 무엇으로 할 거야?"),
  HumanMessage(content="내일은 수영을 가야지!"),
  AIMessage(content="수영이라니, 정말 좋은 운동이야. 수영장은 어디로 다녀?"),
  HumanMessage(content="주말에는 영화를 보러 갈 거야!"),
  AIMessage(content="주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?"),
]

In [ ]:
trimmer = trim_messages(
    max_tokens=80, 
    token_counter=llm,
    strategy="last",
    include_system=True,
    start_on="human"
)


In [20]:
trimmer.invoke(messages)

[SystemMessage(content='사용자의 질문에 2문장 이내로 짧게 대답해', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='내일은 수영을 가야지!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='수영이라니, 정말 좋은 운동이야. 수영장은 어디로 다녀?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='주말에는 영화를 보러 갈 거야!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?', additional_kwargs={}, response_metadata={})]

In [26]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

chain = trimmer | llm | parser

In [35]:
messages = [
  SystemMessage(content="사용자의 질문에 2문장 이내로 짧게 대답해"),
  HumanMessage(content="오늘은 떡볶이를 먹어야지!"),
  AIMessage(content="정말 좋은 생각이야. 음료는 무엇으로 할 거야?"),
  HumanMessage(content="내일은 수영을 가야지!"),
  AIMessage(content="수영이라니, 정말 좋은 운동이야. 수영장은 어디로 다녀?"),
  HumanMessage(content="주말에는 영화를 보러 갈 거야!"),
  AIMessage(content="주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?"),
  HumanMessage(content="오늘 뭘 먹는다고 했지?")
]

chain.invoke(messages)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableSequence > chain:trim_messages] Entering Chain run with input:
[inputs]
[chain/end] [chain:RunnableSequence > chain:trim_messages] s] Exiting Chain run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: 사용자의 질문에 2문장 이내로 짧게 대답해\nHuman: 내일은 수영을 가야지!\nAI: 수영이라니, 정말 좋은 운동이야. 수영장은 어디로 다녀?\nHuman: 주말에는 영화를 보러 갈 거야!\nAI: 주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?\nHuman: 오늘 뭘 먹는다고 했지?"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "혼자 고민 중이야. 한 번 더 생각해볼까?",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "

'혼자 고민 중이야. 한 번 더 생각해볼까?'

In [37]:
import langchain
langchain.debug = True

messages = [
  SystemMessage(content="사용자의 질문에 2문장 이내로 짧게 대답해"),
  HumanMessage(content="오늘은 떡볶이를 먹어야지!"),
  AIMessage(content="정말 좋은 생각이야. 음료는 무엇으로 할 거야?"),
  HumanMessage(content="내일은 수영을 가야지!"),
  AIMessage(content="수영이라니, 정말 좋은 운동이야. 수영장은 어디로 다녀?"),
  HumanMessage(content="주말에는 영화를 보러 갈 거야!"),
  AIMessage(content="주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?"),
  HumanMessage(content="주말에 어디를 간다고 했지?")
]

chain.invoke(messages)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableSequence > chain:trim_messages] Entering Chain run with input:
[inputs]
[chain/end] [chain:RunnableSequence > chain:trim_messages] s] Exiting Chain run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: 사용자의 질문에 2문장 이내로 짧게 대답해\nHuman: 내일은 수영을 가야지!\nAI: 수영이라니, 정말 좋은 운동이야. 수영장은 어디로 다녀?\nHuman: 주말에는 영화를 보러 갈 거야!\nAI: 주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?\nHuman: 주말에 어디를 간다고 했지?"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [1.84s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "주말에 영화를 보러 간다고 했었지! 어떤 영화를 볼 계획이야?",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langcha

'주말에 영화를 보러 간다고 했었지! 어떤 영화를 볼 계획이야?'